In [1]:
import pandas as pd
import numpy as np
import ast
import missingno as msno
from geopy.geocoders import Nominatim
import geopandas as gpd
import re
import pycountry_convert as pc
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amekk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\amekk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
import dataframes as RAW
import cleaning as CLEAN

In [3]:
data = pd.read_csv('../Clean data/give_to_ali.csv')

In [4]:
data

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,Map ID,Character ID,Actor ID,Role
0,20663735,/m/051zjwb,2000,M.K. Menon,1935-12-10,M,NaN,/m/0dryh9k,Thilakan,64.0,/m/059t6pp,/m/0h73lnb,/m/02hkvw,Primary
1,20663735,/m/051zjwb,2000,Bharathan,1971-01-01,M,NaN,NaN,Kalabhavan Mani,28.0,/m/059t4pc,/m/0h86g_9,/m/0g6m_k,Missed
2,20663735,/m/051zjwb,2000,Nandagopal Marar,1951-09-07,M,1.78,/m/04mvp8,Mammootty,48.0,/m/059t6pv,/m/0h27qg5,/m/02hkv5,Secondary
3,20663735,/m/051zjwb,2000,Marancheri Induchoodan,1960-05-21,M,1.72,/m/0dryh9k,Mohanlal,39.0,/m/059t6p_,/m/0h8gtfl,/m/02fbpz,Primary
4,8153846,/m/026tk34,1994-03-09,Serge Karamasov,1958-11-24,M,NaN,NaN,Alain Chabat,35.0,/m/02nzwsh,/m/02nzwsk,/m/0krrfy,Primary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,15072401,/m/03hglxv,1936-01-17,Warren T. Phelps,1891-03-21,M,NaN,NaN,Jonathan Hale,NaN,/m/0n4kl7c,/m/0n4kl7g,/m/09_02r,Secondary
449,15072401,/m/03hglxv,1936-01-17,Ulrich,1880-02-18,M,NaN,NaN,Egon Brecher,NaN,/m/0n4kld4,/m/0n4kld7,/m/0c4l_g,Secondary
450,15072401,/m/03hglxv,1936-01-17,Carlotta,NaN,F,NaN,NaN,Gloria Roy,NaN,/m/0n4klht,/m/0n4klhx,/m/0btz18d,Missed
451,33269085,/m/0b3vgz3,2002-11-15,Sunder Kapoor,1970-12-03,M,1.79,/m/0dryh9k,Jimmy Shergill,31.0,/m/0h36mn3,/m/0h36mn6,/m/075r_k,Secondary


- This data lacks features that we want to integrate in our study. To find them, we need to merge with the other features that we want.

In [5]:
merged_data = pd.merge(data, RAW.movie_data_scraped[['Wiki ID', 'Movie name','Revenue','Wins','Nominations','Runtime','Languages','Countries','Genres','IMDb rating']], on='Wiki ID', how='left')
merged_data

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,...,Role,Movie name,Revenue,Wins,Nominations,Runtime,Languages,Countries,Genres,IMDb rating
0,20663735,/m/051zjwb,2000,M.K. Menon,1935-12-10,M,NaN,/m/0dryh9k,Thilakan,64.0,...,Primary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20663735,/m/051zjwb,2000,Bharathan,1971-01-01,M,NaN,NaN,Kalabhavan Mani,28.0,...,Missed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20663735,/m/051zjwb,2000,Nandagopal Marar,1951-09-07,M,1.78,/m/04mvp8,Mammootty,48.0,...,Secondary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20663735,/m/051zjwb,2000,Marancheri Induchoodan,1960-05-21,M,1.72,/m/0dryh9k,Mohanlal,39.0,...,Primary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8153846,/m/026tk34,1994-03-09,Serge Karamasov,1958-11-24,M,NaN,NaN,Alain Chabat,35.0,...,Primary,La Cité de la peur,NaN,NaN,NaN,93.0,"{""/m/064_8sq"": ""French Language""}","{""/m/0f8l9c"": ""France""}","{""/m/0vgkd"": ""Black comedy"", ""/m/06nbt"": ""Sati...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,15072401,/m/03hglxv,1936-01-17,Warren T. Phelps,1891-03-21,M,NaN,NaN,Jonathan Hale,NaN,...,Secondary,Charlie Chan's Secret,NaN,NaN,NaN,72.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/02n4kr"": ""Myster...",7.0
449,15072401,/m/03hglxv,1936-01-17,Ulrich,1880-02-18,M,NaN,NaN,Egon Brecher,NaN,...,Secondary,Charlie Chan's Secret,NaN,NaN,NaN,72.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/02n4kr"": ""Myster...",7.0
450,15072401,/m/03hglxv,1936-01-17,Carlotta,NaN,F,NaN,NaN,Gloria Roy,NaN,...,Missed,Charlie Chan's Secret,NaN,NaN,NaN,72.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/02n4kr"": ""Myster...",7.0
451,33269085,/m/0b3vgz3,2002-11-15,Sunder Kapoor,1970-12-03,M,1.79,/m/0dryh9k,Jimmy Shergill,31.0,...,Secondary,Kehtaa Hai Dil Baar Baar,95415.0,NaN,1.0,119.0,"{""/m/03k50"": ""Hindi Language"", ""/m/02h40lc"": ""...","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/02l7c8"": ""Romance Film"", ""/m/05p553"": ""Co...",4.7


- We can see that for columns `Genres` , `Languages` and `Countries`, we have dictionaries containing the different informations. It is difficult to handle them directly, so we are gonna transform them into lists.

In [7]:
# We create a dataframe containing Wiki IDs and Country names to merge them afterwards with the initial dataframe
filtered_df_countries = merged_data[merged_data['Countries'].notnull()]
filtered_df_countries['Country'] = filtered_df_countries['Countries'].apply(lambda x: re.findall(r'": "(.*?)"', x))
filtered_df_countries = filtered_df_countries[['Wiki ID', 'Country']]
filtered_df_countries

C:\Users\amekk\AppData\Local\Temp\ipykernel_28740\3671660944.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_countries['Country'] = filtered_df_countries['Countries'].apply(lambda x: re.findall(r'": "(.*?)"', x))


,Wiki ID,Country
4,8153846,[France]
5,8153846,[France]
6,8153846,[France]
7,8153846,[France]
8,8153846,[France]
...,...,...
448,15072401,[United States of America]
449,15072401,[United States of America]
450,15072401,[United States of America]
451,33269085,"[United States of America, India]"


In [8]:
# We create a dataframe containing Wiki IDs and Languages to merge them afterwards with the initial dataframe
filtered_df_language = merged_data[merged_data['Languages'].notnull()]
filtered_df_language['Language'] = filtered_df_language['Languages'].apply(lambda x: re.findall(r'": "(.*?)"', x))
filtered_df_language = filtered_df_language[['Wiki ID', 'Language']]
filtered_df_language

C:\Users\amekk\AppData\Local\Temp\ipykernel_28740\4074234960.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_language['Language'] = filtered_df_language['Languages'].apply(lambda x: re.findall(r'": "(.*?)"', x))


,Wiki ID,Language
4,8153846,[French Language]
5,8153846,[French Language]
6,8153846,[French Language]
7,8153846,[French Language]
8,8153846,[French Language]
...,...,...
448,15072401,[English Language]
449,15072401,[English Language]
450,15072401,[English Language]
451,33269085,"[Hindi Language, English Language]"


In [9]:
# We create a dataframe containing Wiki IDs and Genres to merge them afterwards with the initial dataframe
filtered_df_genres = merged_data[merged_data['Genres'].notnull()]
filtered_df_genres['Genre'] = filtered_df_genres['Genres'].apply(lambda x: re.findall(r'": "(.*?)"', x))
filtered_df_genres = filtered_df_genres[['Wiki ID', 'Genre']]
filtered_df_genres

C:\Users\amekk\AppData\Local\Temp\ipykernel_28740\1308941463.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_genres['Genre'] = filtered_df_genres['Genres'].apply(lambda x: re.findall(r'": "(.*?)"', x))


,Wiki ID,Genre
4,8153846,"[Black comedy, Satire, Comedy]"
5,8153846,"[Black comedy, Satire, Comedy]"
6,8153846,"[Black comedy, Satire, Comedy]"
7,8153846,"[Black comedy, Satire, Comedy]"
8,8153846,"[Black comedy, Satire, Comedy]"
...,...,...
448,15072401,"[Thriller, Mystery, Comedy film, Crime Fiction..."
449,15072401,"[Thriller, Mystery, Comedy film, Crime Fiction..."
450,15072401,"[Thriller, Mystery, Comedy film, Crime Fiction..."
451,33269085,"[Romance Film, Comedy film]"


- We merge everything together.

In [10]:
final_data = pd.merge(merged_data, filtered_df_language, on = 'Wiki ID', how='left')
final_data = pd.merge(final_data, filtered_df_genres, on = 'Wiki ID', how = 'left')
final_data = pd.merge(final_data, filtered_df_countries, on = 'Wiki ID', how = 'left')

final_data.drop_duplicates(subset=['Character name', 'Actor name'], inplace=True)

# Drop columns by name
columns_to_drop = ['Languages', 'Countries', 'Genres']  # Replace with the names of the columns you want to drop
final_data.drop(columns=columns_to_drop, inplace=True)
#final_data = final_data.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
final_data

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,...,Role,Movie name,Revenue,Wins,Nominations,Runtime,IMDb rating,Language,Genre,Country
0,20663735,/m/051zjwb,2000,M.K. Menon,1935-12-10,M,NaN,/m/0dryh9k,Thilakan,64.0,...,Primary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20663735,/m/051zjwb,2000,Bharathan,1971-01-01,M,NaN,NaN,Kalabhavan Mani,28.0,...,Missed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20663735,/m/051zjwb,2000,Nandagopal Marar,1951-09-07,M,1.78,/m/04mvp8,Mammootty,48.0,...,Secondary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20663735,/m/051zjwb,2000,Marancheri Induchoodan,1960-05-21,M,1.72,/m/0dryh9k,Mohanlal,39.0,...,Primary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8153846,/m/026tk34,1994-03-09,Serge Karamasov,1958-11-24,M,NaN,NaN,Alain Chabat,35.0,...,Primary,La Cité de la peur,NaN,NaN,NaN,93.0,NaN,[French Language],"[Black comedy, Satire, Comedy]",[France]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243559,15072401,/m/03hglxv,1936-01-17,Warren T. Phelps,1891-03-21,M,NaN,NaN,Jonathan Hale,NaN,...,Secondary,Charlie Chan's Secret,NaN,NaN,NaN,72.0,7.0,[English Language],"[Thriller, Mystery, Comedy film, Crime Fiction...",[United States of America]
1244559,15072401,/m/03hglxv,1936-01-17,Ulrich,1880-02-18,M,NaN,NaN,Egon Brecher,NaN,...,Secondary,Charlie Chan's Secret,NaN,NaN,NaN,72.0,7.0,[English Language],"[Thriller, Mystery, Comedy film, Crime Fiction...",[United States of America]
1245559,15072401,/m/03hglxv,1936-01-17,Carlotta,NaN,F,NaN,NaN,Gloria Roy,NaN,...,Missed,Charlie Chan's Secret,NaN,NaN,NaN,72.0,7.0,[English Language],"[Thriller, Mystery, Comedy film, Crime Fiction...",[United States of America]
1246559,33269085,/m/0b3vgz3,2002-11-15,Sunder Kapoor,1970-12-03,M,1.79,/m/0dryh9k,Jimmy Shergill,31.0,...,Secondary,Kehtaa Hai Dil Baar Baar,95415.0,NaN,1.0,119.0,4.7,"[Hindi Language, English Language]","[Romance Film, Comedy film]","[United States of America, India]"


In [34]:
# Extracting column names from the 'merged' dataframe
feature_names = final_data.columns.tolist()

# Creating a new dataframe with feature names
feature_df = pd.DataFrame({'Feature Names': feature_names})
feature_df

,Feature Names
0,Wiki ID
1,Freebase ID
2,Release date
3,Character name
4,Actor DOB
5,Actor gender
6,Actor height
7,Actor ethnicity
8,Actor name
9,Actor age at release


## One-hot encoding of categorical variables

In [11]:
data_one_hot = final_data.copy()

- We are now going to create columns to one-hot encode every character based on the movie where it stars.

In [13]:
# We explode the datafrme to obtain every genre separated, only linked thanks to the Wiki ID
genre_explode = filtered_df_genres.explode('Genre')
genre_explode

,Wiki ID,Genre
4,8153846,Black comedy
4,8153846,Satire
4,8153846,Comedy
5,8153846,Black comedy
5,8153846,Satire
...,...,...
450,15072401,Horror
451,33269085,Romance Film
451,33269085,Comedy film
452,33269085,Romance Film


In [14]:
# We first select the names of the different "tracks-onehot" columns 
genre_unique = genre_explode['Genre'].unique()
track_names = genre_explode['Genre'].unique()+"-onehot"

genre_unique

array(['Black comedy', 'Satire', 'Comedy', 'Comedy film', 'Period piece',
       'Drama', 'Indie', 'Action/Adventure', 'Natural horror films',
       'Action', 'Horror', 'Thriller', 'Mystery', 'Crime Fiction',
       'Television movie', 'Slapstick', 'Fantasy Comedy', 'Fantasy',
       'Domestic Comedy', 'Docudrama', 'World cinema', 'Courtroom Drama',
       'Romantic comedy', 'Media Satire', 'Americana', 'Comedy-drama',
       'Black-and-white', 'Romance Film', 'Science Fiction',
       'Supernatural', 'Romantic drama', 'B-movie', 'Slasher',
       'Experimental film', 'Musical', 'Musical Drama', 'Avant-garde',
       'Western', 'B-Western', 'Adventure', 'Short Film',
       'Comedy of manners', 'Tragicomedy', 'Martial Arts Film', 'Spy',
       'Cult', 'Coming of age', 'Teen', 'Biopic [feature]', 'Bollywood',
       'Political drama', 'War film', 'History', 'Costume Adventure',
       'Epic', 'Historical Epic', 'Chinese Movies', 'Crime Thriller',
       'Film adaptation', 'Family Film'

### Question: On fait quoi pour les genres qui se ressemblent (Comedy et Comedy film, Fantasy comedy, Fantasy, Western, B-Western)

In [15]:
# We add columns containing track_names, initialized with 0s
for name in track_names:
    data_one_hot[name] = 0

# We make sure that the columns were created
data_one_hot

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,...,Family Film-onehot,Family Drama-onehot,Christian film-onehot,Silent film-onehot,Biography-onehot,Costume drama-onehot,Hagiography-onehot,Historical fiction-onehot,LGBT-onehot,Gay-onehot
0,20663735,/m/051zjwb,2000,M.K. Menon,1935-12-10,M,NaN,/m/0dryh9k,Thilakan,64.0,...,0,0,0,0,0,0,0,0,0,0
1,20663735,/m/051zjwb,2000,Bharathan,1971-01-01,M,NaN,NaN,Kalabhavan Mani,28.0,...,0,0,0,0,0,0,0,0,0,0
2,20663735,/m/051zjwb,2000,Nandagopal Marar,1951-09-07,M,1.78,/m/04mvp8,Mammootty,48.0,...,0,0,0,0,0,0,0,0,0,0
3,20663735,/m/051zjwb,2000,Marancheri Induchoodan,1960-05-21,M,1.72,/m/0dryh9k,Mohanlal,39.0,...,0,0,0,0,0,0,0,0,0,0
4,8153846,/m/026tk34,1994-03-09,Serge Karamasov,1958-11-24,M,NaN,NaN,Alain Chabat,35.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243559,15072401,/m/03hglxv,1936-01-17,Warren T. Phelps,1891-03-21,M,NaN,NaN,Jonathan Hale,NaN,...,0,0,0,0,0,0,0,0,0,0
1244559,15072401,/m/03hglxv,1936-01-17,Ulrich,1880-02-18,M,NaN,NaN,Egon Brecher,NaN,...,0,0,0,0,0,0,0,0,0,0
1245559,15072401,/m/03hglxv,1936-01-17,Carlotta,NaN,F,NaN,NaN,Gloria Roy,NaN,...,0,0,0,0,0,0,0,0,0,0
1246559,33269085,/m/0b3vgz3,2002-11-15,Sunder Kapoor,1970-12-03,M,1.79,/m/0dryh9k,Jimmy Shergill,31.0,...,0,0,0,0,0,0,0,0,0,0


- Now that the columns were created, we are going to iterate over each row and put 1 on the corresponding column if the character plays in a film of the same genre.

In [17]:
# We transform track_names 
track_names_columns = track_names.tolist()

# Iterate over each row and set one-hot encoded columns
for index, row in data_one_hot.iterrows(): # iteration over indexes and rows of data_one_hot
    if row['Genre'] is not np.nan: # condition: if the value is not a Nan, we can enter
        #print(index) # we print to make sure that we enter the loop (à supprimer)
        genres = row['Genre'] # we extract lists of the genre for each row
        for genre in genres: # we check fror the different genres in the list
            #print(genre) # we print the genre (à supprimer)
            if genre +'-onehot' in track_names_columns: # condition if the genre-onehot is present in track_names_colums
                #print('je rentre') # we make sure we enter the loop (à supprimer)
                data_one_hot.at[index, genre+'-onehot'] = 1 

# We display the dataframe
data_one_hot

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,...,Family Film-onehot,Family Drama-onehot,Christian film-onehot,Silent film-onehot,Biography-onehot,Costume drama-onehot,Hagiography-onehot,Historical fiction-onehot,LGBT-onehot,Gay-onehot
0,20663735,/m/051zjwb,2000,M.K. Menon,1935-12-10,M,NaN,/m/0dryh9k,Thilakan,64.0,...,0,0,0,0,0,0,0,0,0,0
1,20663735,/m/051zjwb,2000,Bharathan,1971-01-01,M,NaN,NaN,Kalabhavan Mani,28.0,...,0,0,0,0,0,0,0,0,0,0
2,20663735,/m/051zjwb,2000,Nandagopal Marar,1951-09-07,M,1.78,/m/04mvp8,Mammootty,48.0,...,0,0,0,0,0,0,0,0,0,0
3,20663735,/m/051zjwb,2000,Marancheri Induchoodan,1960-05-21,M,1.72,/m/0dryh9k,Mohanlal,39.0,...,0,0,0,0,0,0,0,0,0,0
4,8153846,/m/026tk34,1994-03-09,Serge Karamasov,1958-11-24,M,NaN,NaN,Alain Chabat,35.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243559,15072401,/m/03hglxv,1936-01-17,Warren T. Phelps,1891-03-21,M,NaN,NaN,Jonathan Hale,NaN,...,0,0,0,0,0,0,0,0,0,0
1244559,15072401,/m/03hglxv,1936-01-17,Ulrich,1880-02-18,M,NaN,NaN,Egon Brecher,NaN,...,0,0,0,0,0,0,0,0,0,0
1245559,15072401,/m/03hglxv,1936-01-17,Carlotta,NaN,F,NaN,NaN,Gloria Roy,NaN,...,0,0,0,0,0,0,0,0,0,0
1246559,33269085,/m/0b3vgz3,2002-11-15,Sunder Kapoor,1970-12-03,M,1.79,/m/0dryh9k,Jimmy Shergill,31.0,...,0,0,0,0,0,0,0,0,0,0


- We want to check if the operation worked, and that the genres were given correctly (we do a simple check on one genre)

In [18]:
sum = genre_explode['Genre'].str.count('Black comedy').sum()
print(sum)
sum1 = data_one_hot['Black comedy-onehot'].sum()
print(sum1)

16
16


- We have the same number of movies for the genre `Black comedy`, so we assume that our code works well.

- We redo the same thing for `Language` and `Country`

In [20]:
language_explode = filtered_df_language.explode('Language')

language_explode

,Wiki ID,Language
4,8153846,French Language
5,8153846,French Language
6,8153846,French Language
7,8153846,French Language
8,8153846,French Language
...,...,...
450,15072401,English Language
451,33269085,Hindi Language
451,33269085,English Language
452,33269085,Hindi Language


In [26]:
language_unique = language_explode['Language'].unique() #les langues uniques 
language_track_names = language_unique+"-onehot" #les langues uniques + onehot à la fin

language_unique
#language_track_names

array(['French Language', 'German Language', 'Vietnamese Language',
       'English Language', 'Russian Language', 'Polish Language',
       'Spanish Language', 'Italian Language', 'Cantonese',
       'Korean Language', 'Hindi Language', 'Czech Language',
       'Bhojpuri Language', 'Standard Mandarin', 'Ukrainian Language',
       'Turkish Language', 'Urdu Language', 'Malayalam Language',
       'Telugu language', 'Japanese Language', 'Mandarin Chinese',
       'Silent film'], dtype=object)

In [25]:
# Ajoutez des colonnes avec les noms de track_names, initialisées à 0
for name in language_track_names:
    data_one_hot[name] = 0

data_one_hot

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,...,Czech Language-onehot,Bhojpuri Language-onehot,Standard Mandarin-onehot,Ukrainian Language-onehot,Turkish Language-onehot,Urdu Language-onehot,Malayalam Language-onehot,Telugu language-onehot,Japanese Language-onehot,Mandarin Chinese-onehot
0,20663735,/m/051zjwb,2000,M.K. Menon,1935-12-10,M,NaN,/m/0dryh9k,Thilakan,64.0,...,0,0,0,0,0,0,0,0,0,0
1,20663735,/m/051zjwb,2000,Bharathan,1971-01-01,M,NaN,NaN,Kalabhavan Mani,28.0,...,0,0,0,0,0,0,0,0,0,0
2,20663735,/m/051zjwb,2000,Nandagopal Marar,1951-09-07,M,1.78,/m/04mvp8,Mammootty,48.0,...,0,0,0,0,0,0,0,0,0,0
3,20663735,/m/051zjwb,2000,Marancheri Induchoodan,1960-05-21,M,1.72,/m/0dryh9k,Mohanlal,39.0,...,0,0,0,0,0,0,0,0,0,0
4,8153846,/m/026tk34,1994-03-09,Serge Karamasov,1958-11-24,M,NaN,NaN,Alain Chabat,35.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243559,15072401,/m/03hglxv,1936-01-17,Warren T. Phelps,1891-03-21,M,NaN,NaN,Jonathan Hale,NaN,...,0,0,0,0,0,0,0,0,0,0
1244559,15072401,/m/03hglxv,1936-01-17,Ulrich,1880-02-18,M,NaN,NaN,Egon Brecher,NaN,...,0,0,0,0,0,0,0,0,0,0
1245559,15072401,/m/03hglxv,1936-01-17,Carlotta,NaN,F,NaN,NaN,Gloria Roy,NaN,...,0,0,0,0,0,0,0,0,0,0
1246559,33269085,/m/0b3vgz3,2002-11-15,Sunder Kapoor,1970-12-03,M,1.79,/m/0dryh9k,Jimmy Shergill,31.0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# We transform track_names 
language_track_names_columns = language_track_names.tolist()

# Iterate over each row and set one-hot encoded columns
for index, row in data_one_hot.iterrows(): # iteration over indexes and rows of data_one_hot
    if row['Language'] is not np.nan: # condition: if the value is not a Nan, we can enter
        #print(index) # we print to make sure that we enter the loop (à supprimer)
        languages = row['Language'] # we extract lists of the genre for each row
        for language in languages: # we check fror the different genres in the list
            #print(genre) # we print the genre (à supprimer)
            if language +'-onehot' in language_track_names_columns: # condition if the genre-onehot is present in track_names_colums
                #print('je rentre') # we make sure we enter the loop (à supprimer)
                data_one_hot.at[index, language+'-onehot'] = 1 

# We display the dataframe
data_one_hot

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,...,Czech Language-onehot,Bhojpuri Language-onehot,Standard Mandarin-onehot,Ukrainian Language-onehot,Turkish Language-onehot,Urdu Language-onehot,Malayalam Language-onehot,Telugu language-onehot,Japanese Language-onehot,Mandarin Chinese-onehot
0,20663735,/m/051zjwb,2000,M.K. Menon,1935-12-10,M,NaN,/m/0dryh9k,Thilakan,64.0,...,0,0,0,0,0,0,0,0,0,0
1,20663735,/m/051zjwb,2000,Bharathan,1971-01-01,M,NaN,NaN,Kalabhavan Mani,28.0,...,0,0,0,0,0,0,0,0,0,0
2,20663735,/m/051zjwb,2000,Nandagopal Marar,1951-09-07,M,1.78,/m/04mvp8,Mammootty,48.0,...,0,0,0,0,0,0,0,0,0,0
3,20663735,/m/051zjwb,2000,Marancheri Induchoodan,1960-05-21,M,1.72,/m/0dryh9k,Mohanlal,39.0,...,0,0,0,0,0,0,0,0,0,0
4,8153846,/m/026tk34,1994-03-09,Serge Karamasov,1958-11-24,M,NaN,NaN,Alain Chabat,35.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243559,15072401,/m/03hglxv,1936-01-17,Warren T. Phelps,1891-03-21,M,NaN,NaN,Jonathan Hale,NaN,...,0,0,0,0,0,0,0,0,0,0
1244559,15072401,/m/03hglxv,1936-01-17,Ulrich,1880-02-18,M,NaN,NaN,Egon Brecher,NaN,...,0,0,0,0,0,0,0,0,0,0
1245559,15072401,/m/03hglxv,1936-01-17,Carlotta,NaN,F,NaN,NaN,Gloria Roy,NaN,...,0,0,0,0,0,0,0,0,0,0
1246559,33269085,/m/0b3vgz3,2002-11-15,Sunder Kapoor,1970-12-03,M,1.79,/m/0dryh9k,Jimmy Shergill,31.0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
sum_language = language_explode['Language'].str.count('French Language').sum()
print(sum_language)
sum = data_one_hot['French Language-onehot'].sum()
print(sum)

40
40


In [31]:
#exploser notre dataframe sur les countries 
country_explode = filtered_df_countries.explode('Country')

#Récupérer les valeurs uniques des countries 
country_unique = country_explode['Country'].unique() #les countries uniques 

#Récupérer les valeurs uniques des countries-onhot 
country_track_names = country_unique+"-onehot" #les countries uniques + onehot à la fin

# Ajoutez des colonnes avec les noms de country_track_names, initialisées à 0
for name in country_track_names:
    data_one_hot[name] = 0


country_track_names_columns = country_track_names.tolist()

# Iterate over each row and set one-hot encoded columns
for index, row in data_one_hot.iterrows(): #parcouri les index de data_final et les lignes
    if row['Country'] is not np.nan: #si la valeur de genre n'est pas un nan on rentre
        #print(index) #on print l'index de la ligne en question
        
        countries = row['Country'] #extrait les genre de la ligne 1 sous forme de liste 
    
        #print(genres) #imprime cette liste

        for country in countries: #on parcourt la liste de genre issu du film 
            #print(country) #Imprimer le genre en question
            if country +'-onehot' in country_track_names_columns: # Si ce genre +onehot est bien dans la list des track_names_columns
                #print('je rentre') #on est sûrs qu'on rentre 
                data_one_hot.at[index, country+'-onehot'] = 1 #si on est bien rentrés on attribue la valeur 1 


data_one_hot #on display 

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,...,Russia-onehot,United Kingdom-onehot,Spain-onehot,Italy-onehot,Puerto Rico-onehot,South Korea-onehot,India-onehot,Czech Republic-onehot,China-onehot,Denmark-onehot
0,20663735,/m/051zjwb,2000,M.K. Menon,1935-12-10,M,NaN,/m/0dryh9k,Thilakan,64.0,...,0,0,0,0,0,0,0,0,0,0
1,20663735,/m/051zjwb,2000,Bharathan,1971-01-01,M,NaN,NaN,Kalabhavan Mani,28.0,...,0,0,0,0,0,0,0,0,0,0
2,20663735,/m/051zjwb,2000,Nandagopal Marar,1951-09-07,M,1.78,/m/04mvp8,Mammootty,48.0,...,0,0,0,0,0,0,0,0,0,0
3,20663735,/m/051zjwb,2000,Marancheri Induchoodan,1960-05-21,M,1.72,/m/0dryh9k,Mohanlal,39.0,...,0,0,0,0,0,0,0,0,0,0
4,8153846,/m/026tk34,1994-03-09,Serge Karamasov,1958-11-24,M,NaN,NaN,Alain Chabat,35.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243559,15072401,/m/03hglxv,1936-01-17,Warren T. Phelps,1891-03-21,M,NaN,NaN,Jonathan Hale,NaN,...,0,0,0,0,0,0,0,0,0,0
1244559,15072401,/m/03hglxv,1936-01-17,Ulrich,1880-02-18,M,NaN,NaN,Egon Brecher,NaN,...,0,0,0,0,0,0,0,0,0,0
1245559,15072401,/m/03hglxv,1936-01-17,Carlotta,NaN,F,NaN,NaN,Gloria Roy,NaN,...,0,0,0,0,0,0,0,0,0,0
1246559,33269085,/m/0b3vgz3,2002-11-15,Sunder Kapoor,1970-12-03,M,1.79,/m/0dryh9k,Jimmy Shergill,31.0,...,0,0,0,0,0,0,1,0,0,0


In [33]:
sum_country = country_explode['Country'].str.count('France').sum()
print(sum_country)
sum_country_1 = data_one_hot['France-onehot'].sum()
print(sum_country_1)

28
28


In [35]:
# One-hot encoding of the 'Actor gender' column
one_hot_encoded_track = pd.get_dummies(data_one_hot['Actor gender'])

# Convert boolean 'one_hot_encoded_track' columns to integers (0s and 1s)
one_hot_encoded_track = one_hot_encoded_track.astype(int)

# Renaming the new columns to {track}-onehot
one_hot_encoded_track.columns = [f"{col}-{'onehot'}" for col in one_hot_encoded_track.columns]

# Concatenate the one-hot encoded columns with the original DataFrame
data_one_hot = pd.concat([data_one_hot, one_hot_encoded_track], axis=1)

data_one_hot

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,...,Spain-onehot,Italy-onehot,Puerto Rico-onehot,South Korea-onehot,India-onehot,Czech Republic-onehot,China-onehot,Denmark-onehot,F-onehot,M-onehot
0,20663735,/m/051zjwb,2000,M.K. Menon,1935-12-10,M,NaN,/m/0dryh9k,Thilakan,64.0,...,0,0,0,0,0,0,0,0,0,1
1,20663735,/m/051zjwb,2000,Bharathan,1971-01-01,M,NaN,NaN,Kalabhavan Mani,28.0,...,0,0,0,0,0,0,0,0,0,1
2,20663735,/m/051zjwb,2000,Nandagopal Marar,1951-09-07,M,1.78,/m/04mvp8,Mammootty,48.0,...,0,0,0,0,0,0,0,0,0,1
3,20663735,/m/051zjwb,2000,Marancheri Induchoodan,1960-05-21,M,1.72,/m/0dryh9k,Mohanlal,39.0,...,0,0,0,0,0,0,0,0,0,1
4,8153846,/m/026tk34,1994-03-09,Serge Karamasov,1958-11-24,M,NaN,NaN,Alain Chabat,35.0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243559,15072401,/m/03hglxv,1936-01-17,Warren T. Phelps,1891-03-21,M,NaN,NaN,Jonathan Hale,NaN,...,0,0,0,0,0,0,0,0,0,1
1244559,15072401,/m/03hglxv,1936-01-17,Ulrich,1880-02-18,M,NaN,NaN,Egon Brecher,NaN,...,0,0,0,0,0,0,0,0,0,1
1245559,15072401,/m/03hglxv,1936-01-17,Carlotta,NaN,F,NaN,NaN,Gloria Roy,NaN,...,0,0,0,0,0,0,0,0,1,0
1246559,33269085,/m/0b3vgz3,2002-11-15,Sunder Kapoor,1970-12-03,M,1.79,/m/0dryh9k,Jimmy Shergill,31.0,...,0,0,0,0,1,0,0,0,0,1


In [42]:
# One-hot encoding of the 'Role' column
one_hot_encoded_track = pd.get_dummies(data_one_hot['Role'])

# Convert boolean 'one_hot_encoded_track' columns to integers (0s and 1s)
one_hot_encoded_track = one_hot_encoded_track.astype(int)

# Renaming the new columns to {track}-onehot
one_hot_encoded_track.columns = [f"{col}-{'onehot'}" for col in one_hot_encoded_track.columns]

# Concatenate the one-hot encoded columns with the original DataFrame
data_one_hot = pd.concat([data_one_hot, one_hot_encoded_track], axis=1)

data_one_hot.reset_index(drop=True)

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,...,M-onehot,Missed-onehot,Primary-onehot,Secondary-onehot,Missed-onehot,Primary-onehot,Secondary-onehot,Missed-onehot,Primary-onehot,Secondary-onehot
0,20663735,/m/051zjwb,2000,M.K. Menon,1935-12-10,M,NaN,/m/0dryh9k,Thilakan,64.0,...,1,0,1,0,0,1,0,0,1,0
1,20663735,/m/051zjwb,2000,Bharathan,1971-01-01,M,NaN,NaN,Kalabhavan Mani,28.0,...,1,1,0,0,1,0,0,1,0,0
2,20663735,/m/051zjwb,2000,Nandagopal Marar,1951-09-07,M,1.78,/m/04mvp8,Mammootty,48.0,...,1,0,0,1,0,0,1,0,0,1
3,20663735,/m/051zjwb,2000,Marancheri Induchoodan,1960-05-21,M,1.72,/m/0dryh9k,Mohanlal,39.0,...,1,0,1,0,0,1,0,0,1,0
4,8153846,/m/026tk34,1994-03-09,Serge Karamasov,1958-11-24,M,NaN,NaN,Alain Chabat,35.0,...,1,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,15072401,/m/03hglxv,1936-01-17,Warren T. Phelps,1891-03-21,M,NaN,NaN,Jonathan Hale,NaN,...,1,0,0,1,0,0,1,0,0,1
449,15072401,/m/03hglxv,1936-01-17,Ulrich,1880-02-18,M,NaN,NaN,Egon Brecher,NaN,...,1,0,0,1,0,0,1,0,0,1
450,15072401,/m/03hglxv,1936-01-17,Carlotta,NaN,F,NaN,NaN,Gloria Roy,NaN,...,0,1,0,0,1,0,0,1,0,0
451,33269085,/m/0b3vgz3,2002-11-15,Sunder Kapoor,1970-12-03,M,1.79,/m/0dryh9k,Jimmy Shergill,31.0,...,1,0,0,1,0,0,1,0,0,1


## Ethnicity 

In [43]:
data_ethnicity = data_one_hot.copy()
map_ethnicity = data_one_hot.copy()

In [44]:
# We load the ethnicities based on Freebase ID

from SPARQLWrapper import SPARQLWrapper, JSON

def get_ethnicity(freebase_id):
    url = "https://query.wikidata.org/sparql"
    query = ("""
    SELECT ?item ?itemLabel WHERE {
      ?item wdt:P646 '"""+ str(freebase_id)+ """'.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    """
    )
    sparql = SPARQLWrapper(url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.query().convert()
        if results["results"]["bindings"]:
            label = results["results"]["bindings"][0]["itemLabel"]["value"]
            return label
        else:
            return "Unknown"
    except Exception as e:
        return np.nan

In [45]:
map_ethnicity['Ethnicity'] = map_ethnicity['Actor ethnicity'].apply(lambda x: get_ethnicity(x))
map_ethnicity = map_ethnicity[['Actor ethnicity', 'Ethnicity']]
map_ethnicity

,Actor ethnicity,Ethnicity
0,/m/0dryh9k,Indian person
1,NaN,Unknown
2,/m/04mvp8,Malayali
3,/m/0dryh9k,Indian person
4,NaN,Unknown
...,...,...
1243559,NaN,Unknown
1244559,NaN,Unknown
1245559,NaN,Unknown
1246559,/m/0dryh9k,Indian person


## Faudra drop les duplicates mais en tout cas on va utiliser ce fichier pour créer un CSV

In [46]:
data_ethnicity['Actor ethnicity'] = data_ethnicity['Actor ethnicity'].apply(lambda x: get_ethnicity(x))
data_ethnicity.reset_index(drop=True)
data_ethnicity

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,...,M-onehot,Missed-onehot,Primary-onehot,Secondary-onehot,Missed-onehot,Primary-onehot,Secondary-onehot,Missed-onehot,Primary-onehot,Secondary-onehot
0,20663735,/m/051zjwb,2000,M.K. Menon,1935-12-10,M,NaN,Indian person,Thilakan,64.0,...,1,0,1,0,0,1,0,0,1,0
1,20663735,/m/051zjwb,2000,Bharathan,1971-01-01,M,NaN,Unknown,Kalabhavan Mani,28.0,...,1,1,0,0,1,0,0,1,0,0
2,20663735,/m/051zjwb,2000,Nandagopal Marar,1951-09-07,M,1.78,Malayali,Mammootty,48.0,...,1,0,0,1,0,0,1,0,0,1
3,20663735,/m/051zjwb,2000,Marancheri Induchoodan,1960-05-21,M,1.72,Indian person,Mohanlal,39.0,...,1,0,1,0,0,1,0,0,1,0
4,8153846,/m/026tk34,1994-03-09,Serge Karamasov,1958-11-24,M,NaN,Unknown,Alain Chabat,35.0,...,1,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243559,15072401,/m/03hglxv,1936-01-17,Warren T. Phelps,1891-03-21,M,NaN,Unknown,Jonathan Hale,NaN,...,1,0,0,1,0,0,1,0,0,1
1244559,15072401,/m/03hglxv,1936-01-17,Ulrich,1880-02-18,M,NaN,Unknown,Egon Brecher,NaN,...,1,0,0,1,0,0,1,0,0,1
1245559,15072401,/m/03hglxv,1936-01-17,Carlotta,NaN,F,NaN,Unknown,Gloria Roy,NaN,...,0,1,0,0,1,0,0,1,0,0
1246559,33269085,/m/0b3vgz3,2002-11-15,Sunder Kapoor,1970-12-03,M,1.79,Indian person,Jimmy Shergill,31.0,...,1,0,0,1,0,0,1,0,0,1


## Revenue with inflation

In [47]:
data_revenue = data_one_hot.copy()
data_revenue.reset_index(drop = True)

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,...,M-onehot,Missed-onehot,Primary-onehot,Secondary-onehot,Missed-onehot,Primary-onehot,Secondary-onehot,Missed-onehot,Primary-onehot,Secondary-onehot
0,20663735,/m/051zjwb,2000,M.K. Menon,1935-12-10,M,NaN,/m/0dryh9k,Thilakan,64.0,...,1,0,1,0,0,1,0,0,1,0
1,20663735,/m/051zjwb,2000,Bharathan,1971-01-01,M,NaN,NaN,Kalabhavan Mani,28.0,...,1,1,0,0,1,0,0,1,0,0
2,20663735,/m/051zjwb,2000,Nandagopal Marar,1951-09-07,M,1.78,/m/04mvp8,Mammootty,48.0,...,1,0,0,1,0,0,1,0,0,1
3,20663735,/m/051zjwb,2000,Marancheri Induchoodan,1960-05-21,M,1.72,/m/0dryh9k,Mohanlal,39.0,...,1,0,1,0,0,1,0,0,1,0
4,8153846,/m/026tk34,1994-03-09,Serge Karamasov,1958-11-24,M,NaN,NaN,Alain Chabat,35.0,...,1,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,15072401,/m/03hglxv,1936-01-17,Warren T. Phelps,1891-03-21,M,NaN,NaN,Jonathan Hale,NaN,...,1,0,0,1,0,0,1,0,0,1
449,15072401,/m/03hglxv,1936-01-17,Ulrich,1880-02-18,M,NaN,NaN,Egon Brecher,NaN,...,1,0,0,1,0,0,1,0,0,1
450,15072401,/m/03hglxv,1936-01-17,Carlotta,NaN,F,NaN,NaN,Gloria Roy,NaN,...,0,1,0,0,1,0,0,1,0,0
451,33269085,/m/0b3vgz3,2002-11-15,Sunder Kapoor,1970-12-03,M,1.79,/m/0dryh9k,Jimmy Shergill,31.0,...,1,0,0,1,0,0,1,0,0,1


In [49]:
import cpi

# Créez une nouvelle colonne "release year"
data_revenue['Release year'] = None

# Parcours du dataframe
for index, row in data_revenue.iterrows():
    release_date = row['Release date']

    # Vérifiez le type de la valeur dans la colonne "release date"
    if pd.notna(release_date) and isinstance(release_date, float):
        # Si c'est un float non-NaN, assignez directement cette valeur à la nouvelle colonne "release year"
        data_revenue.at[index, 'Release year'] = int(release_date)
    elif pd.notna(release_date) and isinstance(release_date, str):
        # Si c'est une chaîne de caractères non-NaN, convertissez-la en datetime et extrayez l'année
        release_date = pd.to_datetime(release_date)
        data_revenue.at[index, 'Release year'] = release_date.year

#We're going to stop both of the dataframe before 1913 to be sure to have the same revenue data 

#Supprimer tous les films sortis avant 1913
data_revenue = data_revenue[data_revenue['Release year'] >= 1913]


# Add  new column with revenue_adjusted with the inflation rate 
data_revenue['Revenue'] = data_revenue.apply(lambda x: cpi.inflate(x['Revenue'], x['Release year']), axis=1)


data_revenue

C:\Users\amekk\AppData\Local\Temp\ipykernel_28740\3728882579.py:1: StaleDataWarning: CPI data is out of date. To accurately inflate to today's dollars, you must run `cpi.update()`.
  import cpi


,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,...,Missed-onehot,Primary-onehot,Secondary-onehot,Missed-onehot,Primary-onehot,Secondary-onehot,Missed-onehot,Primary-onehot,Secondary-onehot,Release year
0,20663735,/m/051zjwb,2000,M.K. Menon,1935-12-10,M,NaN,/m/0dryh9k,Thilakan,64.0,...,0,1,0,0,1,0,0,1,0,2000
1,20663735,/m/051zjwb,2000,Bharathan,1971-01-01,M,NaN,NaN,Kalabhavan Mani,28.0,...,1,0,0,1,0,0,1,0,0,2000
2,20663735,/m/051zjwb,2000,Nandagopal Marar,1951-09-07,M,1.78,/m/04mvp8,Mammootty,48.0,...,0,0,1,0,0,1,0,0,1,2000
3,20663735,/m/051zjwb,2000,Marancheri Induchoodan,1960-05-21,M,1.72,/m/0dryh9k,Mohanlal,39.0,...,0,1,0,0,1,0,0,1,0,2000
4,8153846,/m/026tk34,1994-03-09,Serge Karamasov,1958-11-24,M,NaN,NaN,Alain Chabat,35.0,...,0,1,0,0,1,0,0,1,0,1994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243559,15072401,/m/03hglxv,1936-01-17,Warren T. Phelps,1891-03-21,M,NaN,NaN,Jonathan Hale,NaN,...,0,0,1,0,0,1,0,0,1,1936
1244559,15072401,/m/03hglxv,1936-01-17,Ulrich,1880-02-18,M,NaN,NaN,Egon Brecher,NaN,...,0,0,1,0,0,1,0,0,1,1936
1245559,15072401,/m/03hglxv,1936-01-17,Carlotta,NaN,F,NaN,NaN,Gloria Roy,NaN,...,1,0,0,1,0,0,1,0,0,1936
1246559,33269085,/m/0b3vgz3,2002-11-15,Sunder Kapoor,1970-12-03,M,1.79,/m/0dryh9k,Jimmy Shergill,31.0,...,0,0,1,0,0,1,0,0,1,2002


# DATA CLEAN. Maintenant faire le choix de comment on va garder les rows qui ont des Nan.